In [2]:
!pip install pyrosettacolabsetup
import pyrosettacolabsetup; pyrosettacolabsetup.install_pyrosetta()
import pyrosetta; pyrosetta.init()


┌──────────────────────────────────────────────────────────────────────────────┐
│                                 PyRosetta-4                                  │
│              Created in JHU by Sergey Lyskov and PyRosetta Team              │
│              (C) Copyright Rosetta Commons Member Institutions               │
│                                                                              │
│ NOTE: USE OF PyRosetta FOR COMMERCIAL PURPOSES REQUIRE PURCHASE OF A LICENSE │
│         See LICENSE.PyRosetta.md or email license@uw.edu for details         │
└──────────────────────────────────────────────────────────────────────────────┘
PyRosetta-4 2025 [Rosetta PyRosetta4.MinSizeRel.python311.ubuntu 2025.03+release.1f5080a079a5261122c0e532c46f61a4f7e20df8 2025-01-13T15:37:50] retrieved from: http://www.pyrosetta.org
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.MinSizeRel.python311.ubuntu r390 2025.03+release.1f5080a079 1f50

In [3]:
import pyrosetta
pyrosetta.init()


┌──────────────────────────────────────────────────────────────────────────────┐
│                                 PyRosetta-4                                  │
│              Created in JHU by Sergey Lyskov and PyRosetta Team              │
│              (C) Copyright Rosetta Commons Member Institutions               │
│                                                                              │
│ NOTE: USE OF PyRosetta FOR COMMERCIAL PURPOSES REQUIRE PURCHASE OF A LICENSE │
│         See LICENSE.PyRosetta.md or email license@uw.edu for details         │
└──────────────────────────────────────────────────────────────────────────────┘
PyRosetta-4 2025 [Rosetta PyRosetta4.MinSizeRel.python311.ubuntu 2025.03+release.1f5080a079a5261122c0e532c46f61a4f7e20df8 2025-01-13T15:37:50] retrieved from: http://www.pyrosetta.org
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.MinSizeRel.python311.ubuntu r390 2025.03+release.1f5080a079 1f50

In [4]:
from google.colab import files

# Upload your files
uploaded = files.upload()

# Files uploaded will be saved in the current working directory



Saving 5c3t.pdb to 5c3t (1).pdb
Saving rank5peptide.pdb to rank5peptide (1).pdb


In [5]:
!ls


'5c3t (1).pdb'	 docked_complex.pdb	      google_drive	      rank5peptide.pdb
 5c3t.pdb	 docked_complex_refined.pdb  'rank5peptide (1).pdb'   sample_data


In [6]:
import pyrosetta
from pyrosetta import pose_from_pdb
from pyrosetta.rosetta.protocols.docking import DockMCMProtocol, setup_foldtree
from pyrosetta.rosetta.core.scoring import ScoreFunctionFactory
from pyrosetta.rosetta.utility import vector1_int

# Initialize PyRosetta
pyrosetta.init()

# Load the PD-1 structure (uploaded file)
protein_pose = pose_from_pdb("5c3t.pdb")

# Load the peptide structure (uploaded file)
peptide_pose = pose_from_pdb("rank5peptide.pdb")

# Combine the two poses
from pyrosetta import Pose
complex_pose = Pose()
complex_pose.assign(protein_pose)
complex_pose.append_pose_by_jump(peptide_pose, 1)

print("Protein-peptide complex created.")


┌──────────────────────────────────────────────────────────────────────────────┐
│                                 PyRosetta-4                                  │
│              Created in JHU by Sergey Lyskov and PyRosetta Team              │
│              (C) Copyright Rosetta Commons Member Institutions               │
│                                                                              │
│ NOTE: USE OF PyRosetta FOR COMMERCIAL PURPOSES REQUIRE PURCHASE OF A LICENSE │
│         See LICENSE.PyRosetta.md or email license@uw.edu for details         │
└──────────────────────────────────────────────────────────────────────────────┘
PyRosetta-4 2025 [Rosetta PyRosetta4.MinSizeRel.python311.ubuntu 2025.03+release.1f5080a079a5261122c0e532c46f61a4f7e20df8 2025-01-13T15:37:50] retrieved from: http://www.pyrosetta.org
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.MinSizeRel.python311.ubuntu r390 2025.03+release.1f5080a079 1f50

In [7]:


# Create a vector1_int instance for movable jumps
movable_jumps = vector1_int()
movable_jumps.append(1)

# Set up fold tree for docking
setup_foldtree(complex_pose, "A_B", movable_jumps)

# Perform docking
dock_mcm = DockMCMProtocol()
dock_mcm.apply(complex_pose)

# Save the docked structure
complex_pose.dump_pdb("docked_complex.pdb")
print("Docking complete. Results saved to 'docked_complex.pdb'.")




core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
protocols.docking.DockMCMProtocol: in DockMCMProtocol.apply
protocols.docking.DockMCMProtocol: Using the DockingTaskFactory.
protocols.docking.DockMCMCycle: Setting up defaults for DockMCMCycle:
protocols.docking.DockMCMCycle: ////////////////////////////////////////////////////////////////////////////////
protocols.docking.DockMCMCycle: /// Dockable Jumps: 1                                                        ///
protocols.docking.DockMCMCycle: /// Translation: 0.1                                                         ///
protocols.docking.DockMCMCycle: /// Rotation: 5                                                              ///
protocols.docking.DockMCMCycle: /// Scorefunction:                                                           ///
protocols.docking.DockMCMCycle: ScoreFunction::show():
weights: (f

In [8]:
# Score the docked complex
# Use the standard 'ref2015' scoring function
scorefxn = ScoreFunctionFactory.create_score_function("ref2015")

# Score the docked complex
binding_score = scorefxn(complex_pose)
print(f"Binding Score: {binding_score}")


Binding Score: 121846.41858198775


In [9]:
from pyrosetta.rosetta.protocols.docking import DockingProtocol

# Initialize the full docking protocol
docking_protocol = DockingProtocol()

# Apply docking
docking_protocol.apply(complex_pose)

# Save the docked structure
complex_pose.dump_pdb("docked_complex_refined.pdb")
print("Docking complete with full protocol. Results saved to 'docked_complex_refined.pdb'.")



basic.io.database: Database file opened: scoring/score_functions/InterchainPotential/interchain_env_log.txt
basic.io.database: Database file opened: scoring/score_functions/InterchainPotential/interchain_pair_log.txt
basic.io.database: Database file opened: scoring/score_functions/EnvPairPotential/env_log.txt
basic.io.database: Database file opened: scoring/score_functions/EnvPairPotential/cbeta_den.txt
basic.io.database: Database file opened: scoring/score_functions/EnvPairPotential/pair_log.txt
basic.io.database: Database file opened: scoring/score_functions/EnvPairPotential/cenpack_log.txt
core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
protocols.docking.DockingInitialPerturbation: Reading options...
core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
core.scoring.ScoreFunctionFactory: SCOREFUNCT